# **PhonePe Pulse Data Visualization and Exploration:**

The Phonepe pulse Github repository contains a large amount of data 
related to various metrics and statistics. The goal is to extract this data
and process it to obtain insights and information that can be visualized 
in a user-friendly manner.

In [2]:
import pandas as pd
import numpy as np
import mysql.connector as sql
import os
from git import Repo
import json


from warnings import filterwarnings
filterwarnings("ignore")

## **Repository Cloning**

In [3]:
# Define the repository URL and the directory where it will be cloned
repo_url = "https://github.com/PhonePe/pulse.git"
clone_dir = "./phonepe_pulse"

# Clone the repository
if not os.path.exists(clone_dir):
    print(f"Cloning repository from {repo_url} to {clone_dir}")
    Repo.clone_from(repo_url, clone_dir)
else:
    print(f"Repository already cloned at {clone_dir}")

Repository already cloned at ./phonepe_pulse


In [4]:
file_path = []

for root, dirs, files in os.walk(clone_dir):
    if 'state' in dirs:
        state_dir = os.path.join(root, 'state')
        file_path.append(state_dir.replace('\\', '/'))
        #print(state_dir)
        for state_folder in os.listdir(state_dir):
            #print(state_folder)
            old_path = os.path.join(state_dir, state_folder)
            #print(old_path)
            new_path = os.path.join(state_dir, state_folder.replace('&', 'and').replace('-', '_'))
            #print(new_path)
            os.rename(old_path, new_path)
            

file_path     

['./phonepe_pulse/data/aggregated/insurance/country/india/state',
 './phonepe_pulse/data/aggregated/transaction/country/india/state',
 './phonepe_pulse/data/aggregated/user/country/india/state',
 './phonepe_pulse/data/map/insurance/country/india/state',
 './phonepe_pulse/data/map/insurance/hover/country/india/state',
 './phonepe_pulse/data/map/transaction/hover/country/india/state',
 './phonepe_pulse/data/map/user/hover/country/india/state',
 './phonepe_pulse/data/top/insurance/country/india/state',
 './phonepe_pulse/data/top/transaction/country/india/state',
 './phonepe_pulse/data/top/user/country/india/state']

## **Extracting data from json file**

### **Aggregated Insurance Data**

In [5]:
def get_aggrigate_insurance_data(location):
    aggrigate_insurance_dict = {
                                'state': [],
                                'year' : [],
                                'quarter' : [],
                                'payment_type' : [],
                                'payment_count' : [],
                                'total_amount' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                with open(quarter_path + quarter, 'r') as f:
                    data = json.load(f)
                    for transaction in data['data']['transactionData']:
                        aggrigate_insurance_dict['payment_type'].append(transaction['name'])
                        if transaction['name'] == 'Insurance':
                            for payment in transaction['paymentInstruments']:
                                aggrigate_insurance_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                                aggrigate_insurance_dict['year'].append(year)
                                aggrigate_insurance_dict['quarter'].append('Q'+quarter.replace('.Json', ''))
                                aggrigate_insurance_dict['payment_count'].append(payment['count'])
                                aggrigate_insurance_dict['total_amount'].append(int(payment['amount']))

    return pd.DataFrame(aggrigate_insurance_dict)

location_path = file_path[0]
location_list = os.listdir(location_path)

df_agg_insurance = get_aggrigate_insurance_data(location_path)

df_agg_insurance.sample(10)


,state,year,quarter,payment_type,payment_count,total_amount
146,Goa,2020,Q4,Insurance,614,682364
392,Nagaland,2022,Q4,Insurance,367,700864
530,Uttarakhand,2021,Q2,Insurance,2742,3265717
42,Arunachal Pradesh,2022,Q4,Insurance,613,1464572
357,Meghalaya,2022,Q1,Insurance,337,729211
80,Chandigarh,2020,Q2,Insurance,149,29537
108,Chhattisgarh,2023,Q2,Insurance,10514,16389610
423,Puducherry,2022,Q3,Insurance,1585,2025529
268,Kerala,2023,Q2,Insurance,61032,100620604
221,Jammu and Kashmir,2023,Q3,Insurance,5453,10975874


### **Aggregated Transaction Data**

In [6]:
def get_aggregate_transaction_data(location):
    aggregate_transaction_dict = {
                                'state': [],
                                'year' : [],
                                'quarter' : [],
                                'payment_type' : [],
                                'payment_count' : [],
                                'total_amount' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                with open(quarter_path + quarter, 'r') as f:
                    data = json.load(f)
                    for transaction in data['data']['transactionData']:
                        aggregate_transaction_dict['payment_type'].append(transaction['name'])
                        #if transaction['name'] == 'transaction':
                        for payment in transaction['paymentInstruments']:
                            aggregate_transaction_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                            aggregate_transaction_dict['year'].append(year)
                            aggregate_transaction_dict['quarter'].append('Q'+quarter.replace('.Json', ''))
                            aggregate_transaction_dict['payment_count'].append(payment['count'])
                            aggregate_transaction_dict['total_amount'].append(int(payment['amount']))

    return pd.DataFrame(aggregate_transaction_dict)

location_path = file_path[1]
location_list = os.listdir(location_path)

df_agg_transaction = get_aggregate_transaction_data(location_path)
df_agg_transaction.sample(10)

,state,year,quarter,payment_type,payment_count,total_amount
1340,Gujarat,2022,Q3,Merchant payments,161264265,109018764232
2256,Lakshadweep,2018,Q2,Merchant payments,42,64568
998,Dadra and Nagar Haveli and Daman and Diu,2024,Q1,Financial Services,6748,10924778
2555,Maharashtra,2021,Q1,Peer-to-peer payments,187021609,599213284361
4450,West Bengal,2022,Q1,Merchant payments,97315025,85854730616
3341,Puducherry,2022,Q4,Recharge & bill payments,531963,453692483
70,Andaman and Nicobar Islands,2021,Q3,Peer-to-peer payments,192321,1077787837
1385,Haryana,2018,Q3,Peer-to-peer payments,3753981,14429454356
1427,Haryana,2020,Q3,Recharge & bill payments,12716319,8452802892
1008,Delhi,2018,Q2,Financial Services,205991,66584728


### **Aggregated User Data**

In [7]:
def get_aggregate_user_data(location):
    aggregate_user_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'Brand' : [],
                            'user_count' : [],
                            'Percentage' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)
                    

                try:   
                    users_by_device = data['data']['usersByDevice']
                    for device in users_by_device:
                            aggregate_user_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                            aggregate_user_dict['year'].append(year)
                            aggregate_user_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                            aggregate_user_dict['Brand'].append(device['brand'])
                            aggregate_user_dict['user_count'].append(device['count'])
                            aggregate_user_dict['Percentage'].append(device['percentage'])

                except: 
                     pass
                    

    return pd.DataFrame(aggregate_user_dict)

location_path = file_path[2]
location_list = os.listdir(location_path)

df_agg_user = get_aggregate_user_data(location_path)
df_agg_user.sample(10)

,state,year,quarter,Brand,user_count,Percentage
2788,Jharkhand,2021,Q4,Motorola,92683,0.014051
1937,Gujarat,2019,Q3,Samsung,1784247,0.211672
725,Assam,2021,Q3,Others,369128,0.094202
4995,Puducherry,2021,Q1,Samsung,68574,0.198031
5944,Telangana,2021,Q2,Realme,1259720,0.068811
3031,Kerala,2018,Q4,Realme,82116,0.046728
3721,Madhya Pradesh,2021,Q4,Oppo,2400977,0.130090
1074,Chandigarh,2021,Q1,Motorola,7989,0.019022
2458,Jammu and Kashmir,2018,Q3,Apple,10996,0.032102
3111,Kerala,2020,Q3,OnePlus,117670,0.021079


### **Map Insurance Data**

In [8]:
def get_map_insurance_data(location):
    
    map_insurance_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'latitude' : [],
                            'longitude' : [],
                            'metric' : [],
                            'district' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                    

                try:   
                    map_data = data['data']['data']['data']
                    for map_item in map_data:
                        map_insurance_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        map_insurance_dict['year'].append(year)
                        map_insurance_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        map_insurance_dict['latitude'].append(map_item[0])
                        map_insurance_dict['longitude'].append(map_item[1])
                        map_insurance_dict['metric'].append(map_item[2])
                        map_insurance_dict['district'].append(map_item[3].replace('district', '').title().replace('And ','and ').strip())

                except: 
                     pass
                    

    return pd.DataFrame(map_insurance_dict) 

location_path = file_path[3]
location_list = os.listdir(location_path)

df_map_insurance = get_map_insurance_data(location_path)
df_map_insurance

,state,year,quarter,latitude,longitude,metric,district
0,Andaman and Nicobar Islands,2020,Q2,9.173490,92.812846,3.0,Nicobars
1,Andaman and Nicobar Islands,2020,Q2,11.665257,92.753094,2.0,South Andaman
2,Andaman and Nicobar Islands,2020,Q2,11.665446,92.733193,1.0,South Andaman
3,Andaman and Nicobar Islands,2020,Q3,11.653981,92.743142,8.0,South Andaman
4,Andaman and Nicobar Islands,2020,Q3,11.654075,92.723246,8.0,South Andaman
...,...,...,...,...,...,...,...
828274,West Bengal,2024,Q1,24.683451,88.016741,1.0,Maldah
828275,West Bengal,2024,Q1,23.667646,87.613701,1.0,Birbhum
828276,West Bengal,2024,Q1,21.872267,87.047437,1.0,Paschim Medinipur
828277,West Bengal,2024,Q1,26.707723,89.563912,1.0,Alipurduar


### **Map Insurance-hover Data**

In [9]:
def get_map_hover_data(location):
    
    map_insurance_hover_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'count' : [],
                            'amount' : [],
                            'district' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    map_in_hover_data = data['data']['hoverDataList']
                    for map_hover in map_in_hover_data:
                        map_insurance_hover_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        map_insurance_hover_dict['year'].append(year)
                        map_insurance_hover_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        map_insurance_hover_dict['count'].append(map_hover['metric'][0]['count'])
                        map_insurance_hover_dict['amount'].append(int(map_hover['metric'][0]['amount']))
                        map_insurance_hover_dict['district'].append(map_hover['name'].replace('district', '').title().replace('And ','and ').strip())

                except: 
                     pass
                    

    #return data['data']['hoverDataList'][0]['metric'][0]['amount']#
    return pd.DataFrame(map_insurance_hover_dict) 

location_path = file_path[4]
location_list = os.listdir(location_path)

df_map_insur_hover = get_map_hover_data(location_path)
df_map_insur_hover

,state,year,quarter,count,amount,district
0,Andaman and Nicobar Islands,2020,Q2,3,795,South Andaman
1,Andaman and Nicobar Islands,2020,Q2,3,565,Nicobars
2,Andaman and Nicobar Islands,2020,Q3,1,281,North and Middle Andaman
3,Andaman and Nicobar Islands,2020,Q3,35,13651,South Andaman
4,Andaman and Nicobar Islands,2020,Q3,5,1448,Nicobars
...,...,...,...,...,...,...
11554,West Bengal,2024,Q1,3305,4128424,Nadia
11555,West Bengal,2024,Q1,1435,1935276,Birbhum
11556,West Bengal,2024,Q1,3345,3891064,Purba Medinipur
11557,West Bengal,2024,Q1,3534,4532095,Maldah


### **Map Transaction-hover Data**

In [10]:
location_path = file_path[5]
location_list = os.listdir(location_path)

df_map_transaction_hover = get_map_hover_data(location_path)
df_map_transaction_hover

,state,year,quarter,count,amount,district
0,Andaman and Nicobar Islands,2018,Q1,442,931663,North and Middle Andaman
1,Andaman and Nicobar Islands,2018,Q1,5688,12560249,South Andaman
2,Andaman and Nicobar Islands,2018,Q1,528,1139848,Nicobars
3,Andaman and Nicobar Islands,2018,Q2,825,1317863,North and Middle Andaman
4,Andaman and Nicobar Islands,2018,Q2,9395,23948235,South Andaman
...,...,...,...,...,...,...
18291,West Bengal,2024,Q1,42154481,79030136775,Nadia
18292,West Bengal,2024,Q1,24812300,46400251550,Birbhum
18293,West Bengal,2024,Q1,46212626,92373546681,Purba Medinipur
18294,West Bengal,2024,Q1,42299563,76716033762,Maldah


### **Map User-hover Data**

In [11]:
def get_map_user_hover_data(location):
    
    map_user_hover_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'reg_user' : [],
                            'app_open' : [],
                            'district' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    map_user_hover_data = data['data']['hoverData']
                    for place, map_user in map_user_hover_data.items():
                        map_user_hover_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        map_user_hover_dict['year'].append(year)
                        map_user_hover_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        
                        map_user_hover_dict['reg_user'].append(map_user['registeredUsers'])
                        map_user_hover_dict['app_open'].append(map_user['appOpens'])
                        map_user_hover_dict['district'].append(place.replace('district', '').title().replace('And ','and ').strip())

                except: 
                     pass
                    

    #return data['data']['hoverData'].items()
    return pd.DataFrame(map_user_hover_dict) 

location_path = file_path[6]
location_list = os.listdir(location_path)

df_map_user_hover = get_map_user_hover_data(location_path)
df_map_user_hover

,state,year,quarter,reg_user,app_open,district
0,Andaman and Nicobar Islands,2018,Q1,632,0,North and Middle Andaman
1,Andaman and Nicobar Islands,2018,Q1,5846,0,South Andaman
2,Andaman and Nicobar Islands,2018,Q1,262,0,Nicobars
3,Andaman and Nicobar Islands,2018,Q2,911,0,North and Middle Andaman
4,Andaman and Nicobar Islands,2018,Q2,8143,0,South Andaman
...,...,...,...,...,...,...
18295,West Bengal,2024,Q1,1671140,13679202,Nadia
18296,West Bengal,2024,Q1,1056537,30502875,Birbhum
18297,West Bengal,2024,Q1,1655920,18688405,Purba Medinipur
18298,West Bengal,2024,Q1,1183956,33270738,Maldah


### **Top Insurance Data**

#### **District**

In [12]:
def get_top_insure_district_data(location):
    
    top_insur_state_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'count' : [],
                            'amount' : [],
                            'district' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    top_insur_st_data = data['data']['districts']
                    for top_insurance in top_insur_st_data:
                        top_insur_state_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        top_insur_state_dict['year'].append(year)
                        top_insur_state_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        
                        top_insur_state_dict['count'].append(top_insurance['metric']['count'])
                        top_insur_state_dict['amount'].append(int(top_insurance['metric']['amount']))
                        top_insur_state_dict['district'].append(top_insurance['entityName'].replace('district', '').title().replace('And ','and ').strip())

                except: 
                     pass
                    

    #return data['data']['districts'][0]['metric']['amount']
    return pd.DataFrame(top_insur_state_dict) 


location_path = file_path[7]
location_list = os.listdir(location_path)

# district wise
df_top_insur_district = get_top_insure_district_data(location_path)
df_top_insur_district

,state,year,quarter,count,amount,district
0,Andaman and Nicobar Islands,2020,Q2,3,565,Nicobars
1,Andaman and Nicobar Islands,2020,Q2,3,795,South Andaman
2,Andaman and Nicobar Islands,2020,Q3,35,13651,South Andaman
3,Andaman and Nicobar Islands,2020,Q3,5,1448,Nicobars
4,Andaman and Nicobar Islands,2020,Q3,1,281,North and Middle Andaman
...,...,...,...,...,...,...
4706,West Bengal,2024,Q1,4414,6125287,Paschim Bardhaman
4707,West Bengal,2024,Q1,3797,4800244,Paschim Medinipur
4708,West Bengal,2024,Q1,3534,4532095,Maldah
4709,West Bengal,2024,Q1,3345,3891064,Purba Medinipur


#### **Pincode**

In [13]:
def get_top_insure_pincode_data(location):
    
    top_insur_pincode_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'count' : [],
                            'amount' : [],
                            'pincode' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    top_insur_pincode_data = data['data']['pincodes']
                    for top_insurance in top_insur_pincode_data:
                        top_insur_pincode_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        top_insur_pincode_dict['year'].append(year)
                        top_insur_pincode_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        
                        top_insur_pincode_dict['count'].append(top_insurance['metric']['count'])
                        top_insur_pincode_dict['amount'].append(int(top_insurance['metric']['amount']))
                        top_insur_pincode_dict['pincode'].append(str(top_insurance['entityName']))

                except: 
                     pass
                    

    #return data['data']['pincodes'][0]['entityName']
    return pd.DataFrame(top_insur_pincode_dict) 


location_path = file_path[7]
location_list = os.listdir(location_path)

# Pincode wisee
df_top_insur_pincode = get_top_insure_pincode_data(location_path)
df_top_insur_pincode



,state,year,quarter,count,amount,pincode
0,Andaman and Nicobar Islands,2020,Q2,3,565,744301
1,Andaman and Nicobar Islands,2020,Q2,2,513,744104
2,Andaman and Nicobar Islands,2020,Q2,1,282,744101
3,Andaman and Nicobar Islands,2020,Q3,9,3432,744112
4,Andaman and Nicobar Islands,2020,Q3,7,3948,744105
...,...,...,...,...,...,...
5596,West Bengal,2024,Q1,563,736083,700039
5597,West Bengal,2024,Q1,558,744878,700015
5598,West Bengal,2024,Q1,516,699258,700006
5599,West Bengal,2024,Q1,507,592231,721301


### **Top Transaction Data**

#### **District**

In [15]:
location_path = file_path[8]
location_list = os.listdir(location_path)

# district wise
df_top_tranc_district = get_top_insure_district_data(location_path)
df_top_tranc_district

,state,year,quarter,count,amount,district
0,Andaman and Nicobar Islands,2018,Q1,5688,12560249,South Andaman
1,Andaman and Nicobar Islands,2018,Q1,528,1139848,Nicobars
2,Andaman and Nicobar Islands,2018,Q1,442,931663,North and Middle Andaman
3,Andaman and Nicobar Islands,2018,Q2,9395,23948235,South Andaman
4,Andaman and Nicobar Islands,2018,Q2,1120,3072436,Nicobars
...,...,...,...,...,...,...
7395,West Bengal,2024,Q1,42299563,76716033762,Maldah
7396,West Bengal,2024,Q1,42154481,79030136775,Nadia
7397,West Bengal,2024,Q1,41536962,75162392138,Howrah
7398,West Bengal,2024,Q1,40874089,71315720048,Hooghly


#### **Pincode**

In [16]:
location_path = file_path[8]
location_list = os.listdir(location_path)

# district wise
df_top_tranc_pincode = get_top_insure_pincode_data(location_path)
df_top_tranc_pincode

,state,year,quarter,count,amount,pincode
0,Andaman and Nicobar Islands,2018,Q1,1622,2769297,744101
1,Andaman and Nicobar Islands,2018,Q1,1223,2238041,744103
2,Andaman and Nicobar Islands,2018,Q1,969,3519059,744102
3,Andaman and Nicobar Islands,2018,Q1,685,1298560,744105
4,Andaman and Nicobar Islands,2018,Q1,340,1039715,744104
...,...,...,...,...,...,...
8919,West Bengal,2024,Q1,4490229,7014125089,732125
8920,West Bengal,2024,Q1,4358390,8114395173,700006
8921,West Bengal,2024,Q1,4321119,8497021002,700001
8922,West Bengal,2024,Q1,4265219,9908795047,732201


### **Top User Data**

#### **District**

In [37]:
def get_top_user_pincode_data(location):
    
    top_user_dist_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'reg_user' : [],
                            'district' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    top_user_dist_data = data['data']['districts']
                    for top_user_dist in top_user_dist_data:
                        top_user_dist_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        top_user_dist_dict['year'].append(year)
                        top_user_dist_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        
                        top_user_dist_dict['reg_user'].append(top_user_dist['registeredUsers'])
                        
                        top_user_dist_dict['district'].append(top_user_dist['name'].replace('district', '').title().replace('And ','and ').strip())

                except: 
                     pass
                    

    #return data['data']['districts'][0]['registeredUsers']
    return pd.DataFrame(top_user_dist_dict) 



location_path = file_path[9]
location_list = os.listdir(location_path)

# district wise
df_top_user_district = get_top_user_pincode_data(location_path)
df_top_user_district

,state,year,quarter,reg_user,district
0,Andaman and Nicobar Islands,2018,Q1,5846,South Andaman
1,Andaman and Nicobar Islands,2018,Q1,632,North and Middle Andaman
2,Andaman and Nicobar Islands,2018,Q1,262,Nicobars
3,Andaman and Nicobar Islands,2018,Q2,8143,South Andaman
4,Andaman and Nicobar Islands,2018,Q2,911,North and Middle Andaman
...,...,...,...,...,...
7395,West Bengal,2024,Q1,1737851,Howrah
7396,West Bengal,2024,Q1,1671140,Nadia
7397,West Bengal,2024,Q1,1655920,Purba Medinipur
7398,West Bengal,2024,Q1,1498903,Paschim Medinipur


#### **Pincode**

In [18]:
def get_top_user_pincode_data(location):
    
    top_user_pincode_dict = {'state': [],
                            'year' : [],
                            'quarter' : [],
                            'reg_user' : [],
                            'pincode' : []
                            }

    for state in location_list:
        year_path = location_path + '/' + state + '/'
        year_list = os.listdir(year_path)

        for year in year_list:
            quarter_path = year_path + '/' + year + '/'
            quarter_list = os.listdir(quarter_path)

            for quarter in quarter_list:
                path_json = quarter_path + quarter
                data = pd.read_json(path_json)

                
                try:   
                    top_user_pincode_data = data['data']['pincodes']
                    for top_user_pincode in top_user_pincode_data:
                        top_user_pincode_dict['state'].append(state.replace('_', ' ').title().replace('And ','and '))
                        top_user_pincode_dict['year'].append(year)
                        top_user_pincode_dict['quarter'].append('Q' + quarter.replace('.Json', ''))
                        
                        top_user_pincode_dict['reg_user'].append(top_user_pincode['registeredUsers'])
                        
                        top_user_pincode_dict['pincode'].append(str(top_user_pincode['name']))

                except: 
                     pass
                    

    #return data['data']['pincodes'][0]['registeredUsers']
    return pd.DataFrame(top_user_pincode_dict) 



location_path = file_path[9]
location_list = os.listdir(location_path)

# pincode wise
df_top_user_pincode = get_top_user_pincode_data(location_path)
df_top_user_pincode

,state,year,quarter,reg_user,pincode
0,Andaman and Nicobar Islands,2018,Q1,1608,744103
1,Andaman and Nicobar Islands,2018,Q1,1108,744101
2,Andaman and Nicobar Islands,2018,Q1,1075,744105
3,Andaman and Nicobar Islands,2018,Q1,1006,744102
4,Andaman and Nicobar Islands,2018,Q1,272,744104
...,...,...,...,...,...
8920,West Bengal,2024,Q1,131676,700015
8921,West Bengal,2024,Q1,127917,742304
8922,West Bengal,2024,Q1,127891,721101
8923,West Bengal,2024,Q1,123919,700150


## **Listing of all dataframes**

In [19]:
phonepe_df = [df for df in globals() if isinstance(globals()[df], pd.core.frame.DataFrame) and df.startswith('df_')]

phonepe_df  

['df_agg_insurance',
 'df_agg_transaction',
 'df_agg_user',
 'df_map_insurance',
 'df_map_insur_hover',
 'df_map_transaction_hover',
 'df_map_user_hover',
 'df_top_insur_district',
 'df_top_insur_pincode',
 'df_top_tranc_district',
 'df_top_tranc_pincode',
 'df_top_user_pincode']

In [20]:
for df_list in phonepe_df:
    df = globals()[df_list]
    print(df_list)
    print([df.columns])
    print([df.shape])
    print('\n')

df_agg_insurance
[Index(['state', 'year', 'quarter', 'payment_type', 'payment_count',
       'total_amount'],
      dtype='object')]
[(574, 6)]


df_agg_transaction
[Index(['state', 'year', 'quarter', 'payment_type', 'payment_count',
       'total_amount'],
      dtype='object')]
[(4494, 6)]


df_agg_user
[Index(['state', 'year', 'quarter', 'Brand', 'user_count', 'Percentage'], dtype='object')]
[(6732, 6)]


df_map_insurance
[Index(['state', 'year', 'quarter', 'latitude', 'longitude', 'metric',
       'district'],
      dtype='object')]
[(828279, 7)]


df_map_insur_hover
[Index(['state', 'year', 'quarter', 'count', 'amount', 'district'], dtype='object')]
[(11559, 6)]


df_map_transaction_hover
[Index(['state', 'year', 'quarter', 'count', 'amount', 'district'], dtype='object')]
[(18296, 6)]


df_map_user_hover
[Index(['state', 'year', 'quarter', 'reg_user', 'app_open', 'district'], dtype='object')]
[(18300, 6)]


df_top_insur_district
[Index(['state', 'year', 'quarter', 'count', 'amount

## **Data Wrangling**

### **Data Mining - Geo Coordinates**

In [21]:
df_map_insurance[['state','district','latitude', 'longitude']].sample(3)

,state,district,latitude,longitude
153003,Gujarat,Rajkot,21.998962,70.794156
815122,West Bengal,Darjiling,26.746630,88.406454
149327,Gujarat,Banas Kantha,24.380058,71.625802


In [22]:
# Group by 'district' and calculate the median for 'latitude' and 'longitude'
df_geo_coordinates = df_map_insurance.groupby(['state','district'])[['latitude', 'longitude']].median().reset_index()

df_geo_coordinates.sample(5)

,state,district,latitude,longitude
583,Telangana,Jagtial,18.813723,78.900533
231,Jammu and Kashmir,Kishtwar,33.341419,75.766219
717,West Bengal,Jhargram,22.432813,86.917333
683,Uttar Pradesh,Rampur,28.821817,79.081452
358,Madhya Pradesh,Shahdol,23.492548,81.422404


### **Data Cleaning**

In [32]:
df_geo_coordinates[df_geo_coordinates['state'] == 'Delhi']

,state,district,latitude,longitude
143,Delhi,Central Delhi,28.627467,77.200275
144,Delhi,East Delhi,28.598243,77.315209
145,Delhi,New Delhi Delhi,28.531302,77.108244
146,Delhi,North Delhi,28.794183,77.108244
147,Delhi,North East Delhi,28.723582,77.246263
148,Delhi,North West Delhi,28.714774,76.970067
149,Delhi,Shahdara Delhi,28.636221,77.292233
150,Delhi,South Delhi,28.443875,77.177274
151,Delhi,South East Delhi Delhi,28.514337,77.292233
152,Delhi,South West Delhi,28.557176,76.923973


In [33]:
delhi_index = df_geo_coordinates[df_geo_coordinates['state'] == 'Delhi'].index.tolist()

for i in delhi_index:
    df_geo_coordinates.at[i,'district'] = df_geo_coordinates.at[i,'district']+' Delhi'

### **Merging geo coordinates with other dataframes**

In [34]:
df_top_insur_district = pd.merge(df_top_insur_district, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_top_insur_district.sample(3)

,state,year,quarter,count,amount,district,latitude_x,longitude_x,latitude_y,longitude_y,latitude,longitude
1894,Karnataka,2024,Q1,9295,14666790,Mysuru,12.256254,76.450524,12.256254,76.450524,12.256254,76.450524
4142,Uttarakhand,2022,Q1,1029,1529648,Haridwar,29.852318,77.956732,29.852318,77.956732,29.852318,77.956732
3355,Punjab,2023,Q1,545,1283520,Bathinda,30.181561,74.996854,30.181561,74.996854,30.181561,74.996854


In [35]:
df_top_tranc_district = pd.merge(df_top_tranc_district, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_top_tranc_district.sample(3)

,state,year,quarter,count,amount,district,latitude_x,longitude_x,latitude_y,longitude_y,latitude,longitude
2196,Himachal Pradesh,2023,Q3,6059411,8079309385,Kangra,32.063329,76.218942,32.063329,76.218942,32.063329,76.218942
1276,Chhattisgarh,2022,Q2,4284921,7037763265,Rajnandgaon,21.100643,80.893118,21.100643,80.893118,21.100643,80.893118
2665,Jharkhand,2022,Q4,41821467,63819119475,Ranchi,23.366013,85.318791,23.366013,85.318791,23.366013,85.318791


In [38]:
df_top_user_district = pd.merge(df_top_user_district, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_top_user_district.sample(3)

,state,year,quarter,reg_user,district,latitude,longitude
3917,Manipur,2020,Q4,8651,Ukhrul,25.095386,94.361652
5572,Rajasthan,2023,Q3,1793526,Jodhpur,26.527543,72.904714
1760,Haryana,2018,Q4,153137,Panipat,29.333776,76.923973


In [39]:
df_map_insur_hover = pd.merge(df_map_insur_hover, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_map_insur_hover.sample(3)

,state,year,quarter,count,amount,district,latitude,longitude
2099,Chhattisgarh,2023,Q3,197,442101,Surajpur,23.265357,82.934022
7802,Rajasthan,2020,Q3,264,72793,Ganganagar,29.494050,73.659508
11374,West Bengal,2024,Q1,12088,16681997,Kolkata,22.521021,88.367639


In [40]:
df_map_transaction_hover = pd.merge(df_map_transaction_hover, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_map_transaction_hover.sample(3)

,state,year,quarter,count,amount,district,latitude,longitude
12025,Punjab,2020,Q2,922228,1936659811,Patiala,30.325865,76.369517
4870,Haryana,2022,Q4,16862076,34368965296,Hisar,29.188665,75.812720
3646,Gujarat,2018,Q1,66032,75032118,Amreli,21.452334,71.305238


In [41]:
df_map_user_hover = pd.merge(df_map_user_hover, df_geo_coordinates, on = ['state', 'district'], how = 'inner')
df_map_user_hover.sample(3)

,state,year,quarter,reg_user,app_open,district,latitude,longitude
6775,Karnataka,2020,Q3,407970,11515402,Hassan,12.972376,76.137794
9756,Maharashtra,2023,Q4,436738,22511529,Bhandara,21.109177,79.745596
14119,Tamil Nadu,2023,Q3,1341973,48355746,Tiruppur,10.943722,77.361155


### **Cheching for data redundancy**

In [42]:
for df_list in phonepe_df:
    df = globals()[df_list]
    print(df_list)
    print([df.info()])
    print('\n')
    print(df.isnull().sum())
    print('\n')

df_agg_insurance
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   state          574 non-null    object
 1   year           574 non-null    object
 2   quarter        574 non-null    object
 3   payment_type   574 non-null    object
 4   payment_count  574 non-null    int64 
 5   total_amount   574 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 27.0+ KB
[None]


state            0
year             0
quarter          0
payment_type     0
payment_count    0
total_amount     0
dtype: int64


df_agg_transaction
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4494 entries, 0 to 4493
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   state          4494 non-null   object
 1   year           4494 non-null   object
 2   quarter        4494 non-null   object
 3   p

#### **Dropping rows with null values from all dataframes**

In [43]:
for df_list in phonepe_df:
    df = globals()[df_list]
    df = df.dropna()
    print('Successfully dropped null values from ' + df_list)

Successfully dropped null values from df_agg_insurance
Successfully dropped null values from df_agg_transaction
Successfully dropped null values from df_agg_user
Successfully dropped null values from df_map_insurance
Successfully dropped null values from df_map_insur_hover
Successfully dropped null values from df_map_transaction_hover
Successfully dropped null values from df_map_user_hover
Successfully dropped null values from df_top_insur_district
Successfully dropped null values from df_top_insur_pincode
Successfully dropped null values from df_top_tranc_district
Successfully dropped null values from df_top_tranc_pincode
Successfully dropped null values from df_top_user_pincode


## **Stroing the Dataframes as CSV files**

In [44]:
for df_list in phonepe_df:
    df = globals()[df_list]
    df.to_csv(df_list+'.csv', index = False)
    print('Successfully saved ' + df_list)

Successfully saved df_agg_insurance
Successfully saved df_agg_transaction
Successfully saved df_agg_user
Successfully saved df_map_insurance
Successfully saved df_map_insur_hover
Successfully saved df_map_transaction_hover
Successfully saved df_map_user_hover
Successfully saved df_top_insur_district
Successfully saved df_top_insur_pincode
Successfully saved df_top_tranc_district
Successfully saved df_top_tranc_pincode
Successfully saved df_top_user_pincode


## **Data Warehusing - MySQL**

### **Establishing Connection to MySQL**

In [ ]:
# establishing to MySQL
conn = sql.connect( user='root',
                    password='A@sD#F45',
                    host='localhost'
                    )
if conn:
    print('Connected to MySQL successfully')
else:
    print('Connection Not established')

cursor = conn.cursor()

Connected to MySQL successfully


### **Creating Database**

In [ ]:
cursor.execute('DROP DATABASE IF EXISTS phonepe')

cursor.execute('CREATE DATABASE phonepe')

cursor.execute('USE phonepe')

### **Creating Tables**

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS agg_insurance(
                state VARCHAR(255),
                year YEAR,
                quarter VARCHAR(255),
                payment_type VARCHAR(255),
                payment_count INT,
                total_amount FLOAT
                )''')


cursor.execute('''CREATE TABLE IF NOT EXISTS agg_transaction(
                    state VARCHAR(255),
                    year YEAR,
                    quarter VARCHAR(255),
                    payment_type VARCHAR(255),
                    payment_count INT,
                    total_amount FLOAT                    
                    )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS agg_user(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               brand VARCHAR(255),
               user_count INT,
               percentage FLOAT               
               ) ''')

In [ ]:
# map
cursor.execute('''CREATE TABLE IF NOT EXISTS agg_insurance(
                state VARCHAR(255),
                year YEAR,
                quarter VARCHAR(255),
                district VARCHAR(255),
                latitude FLOAT,
                longitude FLOAT,
                metric FLOAT
                ) ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS map_transaction(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               count INT,
               amount FLOAT,
               district VARCHAR(255),
               latitude FLOAT,
               longitude FLOAT
               ) ''')


cursor.execute('''CREATE TABLE IF NOT EXISTS map_user(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               reg_user INT,
               app_open INT,
               district VARCHAR(255),
               latitude FLOAT,
               longitude FLOAT
               ) ''')

In [ ]:
# top

cursor.execute('''CREATE TABLE IF NOT EXISTS top_insurance_dist(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               count INT,
               amount FLOAT,
               district VARCHAR(255),
               latitude FLOAT,
               longitude FLOAT
               ) ''')


cursor.execute('''CREATE TABLE IF NOT EXISTS top_insure_pincode(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               count INT,
               amount FLOAT,
               pincode VARCHAR(255)
               ) ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS top_tranc_dist(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               count INT,
               amount FLOAT,
               district VARCHAR(255),
               latitude FLOAT,
               longitude FLOAT
               ) ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS top_tranc_pincode(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               count INT,
               amount FLOAT,
               pincode VARCHAR(255)
               ) ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS top_user_district(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               reg_user INT,
               district VARCHAR(255),
               latitude FLOAT,
               longitude FLOAT
               ) ''')


cursor.execute('''CREATE TABLE IF NOT EXISTS top_user_pincode(
               state VARCHAR(255),
               year YEAR,
               quarter VARCHAR(255),
               reg_user INT,
               pincode VARCHAR(255)
               ) ''')


In [ ]:
# Execute the query to get all table names
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = DATABASE();")

# Fetch all results
table_names = cursor.fetchall()

# Print the table names
for table in table_names:
    print(table[0])

agg_insurance
agg_transaction
agg_user
map_transaction
map_user
top_insurance_dist
top_insure_pincode
top_tranc_dist
top_tranc_pincode
top_user_district
top_user_pincode


### **Loading Data to MySQL**

In [ ]:
def data_to_warehouse(conn, cursor, dfs, table_column):
    try:
        for table_name in dfs.keys():
            df = dfs[table_name]
            columns = table_column[table_name]
            place_holders = ', '.join(['%s'] * len(columns))
            query = f'INSERT INTO {table_name} ({", ".join(columns)}) VALUES ({place_holders})'

            data_to_insert = [tuple(row[column] for column in columns) for _, row in df.iterrows()]
            cursor.executemany(query, data_to_insert)

            conn.commit()
        print("Data Warehousing to MYSQL Completed")
    except Exception as e:
        conn.rollback()
        print(f"An error occurred: {e}")


In [ ]:
dfs = {'agg_insurance': df_agg_insurance,
        'agg_transaction': df_agg_transaction,
        'agg_user':df_agg_user,
        'map_transaction':df_map_transaction_hover,
        'map_user':df_map_user_hover,
        'top_insurance_dist':df_top_insur_district,
        'top_insure_pincode':df_top_insur_pincode,
        'top_tranc_dist':df_top_tranc_district,
        'top_tranc_pincode':df_top_tranc_pincode,
        'top_user_district':df_top_user_district,
        'top_user_pincode': df_top_user_pincode
        }



In [ ]:
table_column = {'agg_insurance': list(df_agg_insurance.columns),
                'agg_transaction': list(df_agg_transaction.columns),
                'agg_user': list(df_agg_user.columns),
                'map_transaction': list(df_map_transaction_hover.columns),
                'map_user': list(df_map_user_hover.columns),
                'top_insurance_dist': list(df_top_insur_district.columns),
                'top_insure_pincode': list(df_top_insur_pincode.columns),
                'top_tranc_dist': list(df_top_tranc_district.columns),
                'top_tranc_pincode': list(df_top_tranc_pincode.columns),
                'top_user_district': list(df_top_user_district.columns),
                'top_user_pincode': list(df_top_user_pincode.columns)
                }

In [ ]:
data_to_warehouse(conn, cursor, dfs, table_column)

Data Warehousing to MYSQL Completed
